# Overview of Day 3
* Preliminaries
* Describing and summarizing data
* Visualization with matplotlib, pandas and seaborn
* Statistical analysis in SciPy and statsmodels

# Preliminaries
* Import stuff
* Load the preprocessed data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm

%matplotlib inline

# Disable annoying SettingWithCopyWarning
pd.options.mode.chained_assignment = None

In [ ]:
# If you don't have the data locally, use the commented line instead
data = pd.read_csv('../data/preprocessed_data.csv')
# data = pd.read_csv('https://raw.githubusercontent.com/tyarkoni/SSI2017/master/data/preprocessed_data.csv', encoding='utf8')  

data.head()

# Describing and summarizing data
* It's a good idea to explore your data before you start modeling
* Surprising how often interesting results reflect overlooked problems

### A bird's eye view
* Simple pandas tools for describing/inspecting entire datasets
    * .describe(), .head() and .tail(), .dtypes(), etc...

In [ ]:
# Describe the continuous variables in the dataset.
# We can also include categoricals, or select other percentiles.
data.describe(percentiles=[0.1, 0.9]).round(2)

## Frequency counts
* 1-dimensional frequency counts can provide quick insight, diagnose problems
* What are the [top dog and cat names](http://dogtime.com/dog-names/female-dog-names/21032-top-10-most-popular-dog-and-cat-names-of-2014) in Austin?

In [ ]:
top_n = 20
for species in ['Dog', 'Cat']:
    name_counts = data.query('animal == @species')['name'].value_counts()
    print("\nTop {} {} names:\n{}".format(top_n, species, name_counts[:top_n]))

### What the heck kind of name is "X"?

In [ ]:
data.query('name == "X"').head(10)

### Contingency tables
* It's useful to examine contingency tables (or crosstabs) for pairs of variables
* Normalized crosstabs can be particularly informative

In [ ]:
# Select only dogs
dogs = data.query('animal=="Dog"')

# Keep only common breeds--estimates won't be reliable for uncommon ones
dogs = dogs.groupby('breed').filter(lambda x: len(x) >= 100)

# How many breeds left?
print("Unique breeds:", dogs['breed'].nunique())

# Generate contingency table
ctab = pd.crosstab(dogs['breed'], data['outcome'])

# Sort in descending order of number of animals of each breed
order = ctab.sum(1).sort_values(ascending=False).index
ctab = ctab.loc[order, :]

# Show just the top 10
ctab.iloc[:10, :]

#### Normalized crosstabs
We can normalize the crosstab by dividing by the sum of the rows and/or columns

In [ ]:
# Normalize by row to show proportion of animals accounted for by each outcome
nctab = pd.crosstab(dogs['breed'], data['outcome'], normalize='index')
nctab = nctab.round(2)  # Round to 2 decimal places for clarity
nctab.iloc[:10, :]

Let's sort by specific outcomes to see, e.g., which breeds are euthanized or adopted most often.

In [ ]:
# Top 10 breeds by euthanasia rate
nctab.sort_values('Euthanasia', ascending=False).iloc[:10]

#### Drilling down further
* Euthanasia is a pretty general category
* Why are Chow Chows and Pit bulls being euthanized at such a high rate?
* Repeat with outcome subtype

In [ ]:
# We can do the same thing for outcome subtype
data['outcome_subtype'] = data['outcome_subtype'].fillna('Unknown')
sctab = pd.crosstab(dogs['breed'], data['outcome_subtype'], normalize='index').round(2)

# We could add a column for number of dogs in each breed
sctab['num_dogs'] = dogs.groupby('breed')['animal'].count()

# sort in descending order of 'aggressive'
sctab.sort_values('Aggressive', ascending=False)[:10]

### What does this mean?
~1 in 8 Pit Bulls that pass through AAC are euthanized (and 1 in 7 Chow Chows!). Some possible interpretations:
* Pit bulls are much more dangerous than other dog breeds
    * A genetic disposition?
    * Abusive owners?
* There could be a bias against pit bulls, causing many to be unnecessarily euthanized
* Mixed-breed dogs with behavioral problems may be more likely to be labeled pit bulls
* Data artifact (e.g., smaller samples for these breeds)
* Others?

# Visualization in Python
* That thing about a picture being worth...
* Python has a rich visualization ecosystem
* The primary platform for visualization in Python is [matplotlib](http://matplotlib.org/)
* Most other visualization platforms build on matplotlib
    * But there are others--e.g., [Altair](https://altair-viz.github.io/), [Bokeh](http://bokeh.pydata.org/en/latest/), [Plotly](https://plot.ly/python/)...


## Matplotlib
* Highly object-oriented (contrast with, e.g., R)
    * Easy to modify/customize plots created in different packages
* Documentation is comprehensive, though not always very well organized
    * The [visualization chapter](http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/04.00-Introduction-To-Matplotlib.ipynb) in Jake Vanderplas's [book](https://github.com/jakevdp/PythonDataScienceHandbook/blob/599aa0fe3f882c0001670e676e5a8d43b92c35fc/notebooks/Index.ipynb) is spectacular
    * A good way to learn is to find a plot similar to what you want in the [matplotlib gallery](https://matplotlib.org/gallery.html#) and modify it
* Provides a basic set of high-level plots (barplot, scatter, etc.)
* Beyond that, API has a fairly steep learning curve

### Simple examples

#### Scatterplot

In [ ]:
# Scatterplot of minimum breed weight vs. height
plt.scatter(data['min_weight'], data['min_height'], color='green', s=30)

# Add axis labels
plt.xlabel("Min. breed weight")
plt.ylabel("Min. breed height")
# Add title
plt.title("Breed height vs. weight", fontsize=16);

#### Line plot
* Let's plot the frequency of each outcome by year (2013 - 2019)...

In [ ]:
# We can use one of the predefined styles, which will completely
# change the way plots look. Try plt.style.available to see
# what's available. For kicks, you can also try plt.xkcd().
plt.style.use('fivethirtyeight')

# Get a list of unique outcomes
outcomes = data['outcome'].unique()

# Loop over outcomes and plot a line for each one
for oc in outcomes:
    # Get data subset for only this outcome
    oc_data = data.query('outcome==@oc')
    # Count number of events in each year
    oc_count_by_year = oc_data.groupby('year')['outcome'].count()
    # Plot a line--note the addition of the label, which will be
    # used by the legend later.
    plt.plot(oc_count_by_year, label=oc)


# Add the legend and position it outside the box
plt.legend(bbox_to_anchor=(1.03, 1.03))

# Other minor touches...
plt.ylabel('Number of events')  # y-axis label
plt.xlim(2013, 2019) # limit x-axis to available years

#### Heatmap

In [ ]:
# Get rid of any style—restores the defaults
plt.rcdefaults()

# A heatmap of the crosstab we made earlier
plt.imshow(ctab[:10], cmap='Blues')
plt.yticks(np.arange(10), ctab[:10].index)
plt.xticks(np.arange(8), ctab.columns, rotation=90);
plt.colorbar();

### More complex examples

#### Stacked bar plot
* Let's write some custom code to generate a nice stacked bar plot
* Later, we'll see how to do this with much less code

In [ ]:
# To prevent clutter, let's show bars only for breeds with > 200 occurrences
min200 = dogs.groupby('breed').filter(lambda x: len(x) >= 200)

ct = pd.crosstab(min200['breed'], min200['outcome'], normalize='index')

n_breeds = ct.shape[0]
breed_x = np.arange(n_breeds)
labels = ct.columns
n_bins = len(labels)

# Set up figure
plt.figure(figsize=(16, 6))

# Set up colors--we use seaborn's color_palette utility for this
colors = sns.color_palette('Set2', n_bins)

# Plot the first color
plt.bar(breed_x, ct.iloc[:, 0], label=labels[0], color=colors[0])

# Plot the other colors
for i in range(1, len(labels)):
    bottom = ct.iloc[:, :(i-1)].sum(1)
    plt.bar(breed_x, ct.iloc[:, i], label=labels[i],
            bottom=bottom, color=colors[i])
    
# Add x-axis labels
plt.xticks(np.arange(n_breeds), ct.index, rotation=90, fontsize=16);

# Add the legend outside the plot.
plt.legend(bbox_to_anchor=(1, 1), fontsize=20);

# # A slightly more complicated version if we want to reverse the legend order
# # to match the bars more closely. The handles stuff is just to reverse the
# # label order to match the bars
# handles, labels = plt.gca().get_legend_handles_labels()
# plt.legend(handles[::-1], labels[::-1], bbox_to_anchor=(1, 1), fontsize=20,);

### Customization in matplotlib
* matplotlib is infinitely customizable
* As in most modern plotting environments, you can do virtually anything
* You just have to be willing to spend enough time on it

#### Matplotlib styles
<img src="https://raw.githubusercontent.com/rasbt/matplotlib-gallery/master/images/formatting_4.png">
https://twitter.com/rasbt/status/731205324187795457

In [ ]:
# A whole bunch of custom plotting code, just to generate
# a relatively simple result.
def plot_outcomes_by_time(data, unit='hour', min_unit=None, max_unit=None,
                          animal=None, panel=None, agg_func='count',
                          error_bars=False):
    ''' Custom plotting functions that displays the number of
    outcomes of each type as a function of a unit of time and
    (optionally) any categorical variable.
    
    Args:
        data (DataFrame): The data to plot
        unit (str): The type of unit to plot on the x-axis
        min_unit (int): Optional lower bound value for unit (e.g.,
            if unit='hour' and min_unit=8, only plots hours > 8)
        max_unit (int): Optional upper bound value for unit
        animal (list): Optional list of animal species to include
        panel (str): Optional name of variable to panel by
        agg_func (str): Name of function used to aggregate within cell
    '''
    if animal is not None:
        data = data[data['animal'].isin(animal)]
    if panel is not None:
        if panel == 'year':
            panel_vars = [2014, 2015, 2016, 2017, 2018]
        else:
            panel_vars = data[panel].unique()
        n_panels = len(panel_vars)
        fig, axes = plt.subplots(1, n_panels, figsize=(5*n_panels, 4),
                                 sharex=True, sharey=True)
    else:
        axes = [plt.gca()]

    outcomes = ['Adoption', 'Transfer', 'Return to Owner', 'Euthanasia']
    dummies = pd.get_dummies(data['outcome'])
    data = pd.concat([data, dummies], axis=1)
    
    if min_unit is not None:
        data = data[data[unit] >= min_unit]
    if max_unit is not None:
        data = data[data[unit] <= max_unit]
    
    for oc in outcomes:
        groupers = [unit]
        if panel is not None:
            groupers.append(panel)
        line = data.query('outcome==@oc')\
            .groupby(groupers)[oc].agg(agg_func).reset_index()
        if panel is not None:
            for i, pan in enumerate(panel_vars):
                pan_line = line[line[panel]==pan]
                axes[i].plot(pan_line[unit], pan_line[oc], label=oc, lw=3)
                axes[i].set_title('{}: {}'.format(panel, pan), fontsize=18)
                axes[i].set_xlabel(unit, fontsize=18)
                axes[i].set_ylabel("No. of outcomes", fontsize=18)
    plt.legend(fontsize=16, loc='best')

In [ ]:
# Now we can flexibly call our custom plotter

# Plot outcome counts by hour, paneling by animal
plot_outcomes_by_time(data, unit='hour', panel='animal')

# Plot outcome counts by day of week, only for dogs, paneling by year
# plot_outcomes_by_time(data, animal=['Dog'], unit='day', panel='year')

# Feel free to experiment...

### Alternative interfaces to matplotlib
* Most of the time you don't want to spend three hours making a nice barplot
* The appeal of packages like ggplot2 (in R) is power and elegance with speed
* A number of high-level Python plotting packages built on matplotlib exist
* For anything built on MPL, you can always customize any plot after the fact

### ggplot2 for Python
* ggplot2 has been (mostly) ported to Python as [plotnine](https://github.com/has2k1/plotnine)
* Syntax is very similar to ggplot2 in R

In [ ]:
# Grouped bar plot with animal and outcome on x-axis and age on y-axis
from plotnine import *
ggplot(data, aes(x='animal', y='age', fill='outcome')) + geom_bar(stat='identity', position='dodge')

### Plotting in pandas
* Pandas provides basic plotting functionality
* Well-integrated with DataFrames
* Often the easiest way to quickly explore the data

#### Small multiples

In [ ]:
# To keep things simple, let's select only breeds with > 300 observations
many_dogs = dogs.groupby('breed').filter(lambda x: len(x)>300)

# Plot age histograms for all remaining breeds--with shared x-axis
# to facilitate comparisons between breeds. Try setting sharey as well!
many_dogs['age'].hist(by=many_dogs['breed'], bins=20, sharex=True);

# Embiggen the figure a bit
plt.gcf().set_size_inches((16, 16))

#### Stacked bars, the easy way

In [ ]:
# Our normalized cross-tab from earlier, with only breeds that
# have a minimum of 200 observations
min200 = dogs.groupby('breed').filter(lambda x: len(x) >= 200)
ct = pd.crosstab(min200['breed'], min200['outcome'], normalize='index')

# get nice colors from seaborn
colors = sns.color_palette('Set2', n_bins)

# the same result as before, in pandas, with much less code
ct.plot.bar(stacked=True, fontsize=14, color=colors)

# note we can still customize in matplotlib after plotting the
# base plot with pandas
plt.xticks(rotation=45, ha='right')

# # # Add the legend outside the plot
plt.legend(bbox_to_anchor=(1, 1), fontsize=16)

# # # Resize everything
plt.gcf().set_size_inches((12, 6))

# # # Ruh-roh! Ugly colors! Fortunately, we can manually pass in a list of
# # # colors to use via the colors argument to plot.bar(). We could reuse
# # # the color list we created earlier in seaborn, for example.

## Plotting in seaborn
* Seaborn is a high-level plotting library based on matplotlib
* Designed to produce attractive figures in very little code
* _Great_ [documentation](https://stanford.edu/~mwaskom/software/seaborn/index.html)
* Most seaborn plotting functions accept pandas DataFrames
* Complete customization of seaborn plots is possible using matplotlib

### Heatmaps

In [ ]:
# The same contingency table from earlier.

ax = sns.heatmap(ctab[:12], annot=True, fmt='d', linewidths=.5,
                 annot_kws={'size': 14}, cmap='Blues')

# Customize the label size and figure size in base matplotlib.
# Notice we can access all axis and figure properties through
# # the handle we saved when calling seaborn.
ax.tick_params(labelsize=14)
ax.figure.set_size_inches((7, 8))  # update the plot size

#### Pairwise correlations between all numeric variables in the dataset

In [ ]:
# Plot a heatmap of pairwise correlations between all continuous variables
r = data.query('animal=="Dog"').corr().round(2)

with sns.plotting_context("notebook", font_scale=1.1):
    ax = sns.heatmap(r, annot=True)
    ax.figure.set_size_inches((12, 9.5))

* Don't like the color palette (`cmap` argument)? Pick a [different one](https://matplotlib.org/examples/color/colormaps_reference.html)!
* Looks like the weight and height variables are almost completely redundant
* Why do month and year show a strong negative correlation (-0.25)?
* Illustrates the need for caution
    * Most "interesting" findings turn out to have banal explanations!

### Pair plots
* Systematic inspection of pairwise relationships is very useful
* We've established that some breeds have very different outcomes
* How do outcomes relate to one another at the breed level?
    * Do some outcomes trade off against others?

In [ ]:
# Breed-level correlations between different outcomes and traits

# Restrict to breed with minimum 50 dogs and outcomes with min. 200 occurrences
tmp_data = data.query('animal=="Dog"')
tmp_data = tmp_data.groupby('outcome').filter(lambda x: len(x) >= 200)
tmp_data = tmp_data.groupby('breed').filter(lambda x: len(x) >= 50)

# Dummy-code the outcomes as binary indicators and concatenate with original data
outcomes = pd.get_dummies(tmp_data['outcome'])
tmp_data = pd.concat([tmp_data, outcomes], axis=1)

# Group by breed and calculate means
tmp_data = tmp_data.groupby('breed').mean().dropna().reset_index()
tmp_data = tmp_data[['Adoption', 'Euthanasia', 'Return to Owner', 'Transfer']]
dog_breed_means = tmp_data  # Save for later

# The actual plotting code is just this
with sns.plotting_context("notebook", font_scale=1.3):  # Make text bigger
    sns.pairplot(tmp_data, diag_kind='hist', kind='reg')

### Bar plots
* Let's explore how outcomes differ as a function of the variables in our dataset
* We'll start by visualizing the raw counts for different animals

In [ ]:
# Probably no point in keeping livestock and birds!
tmp_data = data.query('animal in ["Cat", "Dog", "Other"]')

# And also, let's get rid of outcomes with fewer than 500 entries
# Remember this filtering step from Day 2?
tmp_data = tmp_data.groupby('outcome').filter(lambda x: len(x) >= 500)

g = sns.catplot(x='outcome', data=tmp_data, col='animal', kind='count')
g.set_xticklabels(rotation=45);

The previous plot is kind of useful, so let's abstract it into a method. Then we can do the same thing (even more) quickly for other combinations of variables.

In [ ]:
def aac_factorplot(data, x, facet, y=None, kind='count'):
    g = sns.factorplot(x=x, data=data, y=y, col=facet, kind=kind)
    g.set_xticklabels(rotation=90)

# Let's restrict analysis to only full-year data.
years = [2014, 2015, 2016, 2017, 2018]
full_years = tmp_data.query('year in @years')

aac_factorplot(full_years, 'outcome', 'year')

#### Do temporal variables matter?
* We might expect day of the week to matter

In [ ]:
tmp_data = data[data['animal'].isin(['Dog', 'Cat'])]
tmp_data = tmp_data.groupby('outcome').filter(lambda x: len(x) >= 1000)
sns.factorplot(x='animal', data=tmp_data, hue='outcome', col='day', kind='count') ;

#### What about age?

In [ ]:
tmp_data = data[data['animal'].isin(['Dog', 'Cat'])]
tmp_data = data.query('year > 2013 and year < 2019')
tmp_data = tmp_data.groupby('outcome').filter(lambda x: len(x) >= 500)
sns.factorplot(x='animal', y='age', data=tmp_data, hue='outcome', col='year', kind='bar');

#### Color?

In [ ]:
# Restrict to just dogs
tmp_data = data[data['animal'].isin(['Dog'])]

# Filter on colors with more than 200 rows and outcomes with more than 500
tmp_data = tmp_data.groupby('first_color').filter(lambda x: len(x) >= 200)
tmp_data = tmp_data.groupby('outcome').filter(lambda x: len(x) >= 500)

# Plot!
sns.factorplot(x='outcome', data=tmp_data, hue='first_color', kind='count')
plt.gcf().set_size_inches((12, 4))

# It would be nice if the colors matched the labels, but that would be more work...

### What about uncertainty?
* So far we've almost completely ignored estimation uncertainty
* Good visualization represents uncertainty in a clear way
* E.g., error bars, comparison distributions, etc.

#### Breed age with error bars in matplotlib

In [ ]:
# Group by breed and get mean, standard deviation, and count for each breed
age_data = data.groupby('breed_name')['age'].aggregate(['mean', 'std', 'count'])
age_data['95ci'] = age_data['std']/np.sqrt(age_data['count'])

# Let's pick 30 random breeds, otherwise the plot will be too big
select_inds = np.random.choice(age_data.index, size=30)
age_data = age_data.loc[select_inds, :]

# Sort by ascending mean age
age_data = age_data.sort_values('mean')

# Rotate and align labels. Maybe place them at 45 degrees?
plt.xticks(rotation='vertical', fontsize=12)

# Bar plot with error bars
plt.bar(x=age_data.index, height=age_data['mean'], yerr=age_data['95ci'])

# Add y-axis label and title
plt.ylabel("Mean age (months)", fontsize=16)

# Title
plt.title("Mean age of AAC dogs, by breed", fontsize=18)

# Resize for legibility
plt.gcf().set_size_inches(12, 4)

#### Animal age with bootstrapped CIs in seaborn
* Let's estimate the mean age of different dog breeds, broken down by outcome and day of week
* By default, many seaborn plotting commands will bootstrap 95% confidence intervals

In [ ]:
# Only dogs, cats, and other; the rest have too few observations
tmp_data = data.query('animal=="Dog"')

# Only the most common breeds
tmp_data = tmp_data.groupby('breed').filter(lambda x: len(x) >= 1000)

# Drop infrequent outcomes
tmp_data = tmp_data.groupby('outcome').filter(lambda x: len(x) >= 500)

# Display age in years
tmp_data['age'] /= 12.0

# Plot mean age by outcome on x-axis and separate columns for breeds
g = sns.factorplot(x='outcome', data=tmp_data, y='age', col='breed', )
g.set_xticklabels(rotation=90, fontsize=16);

# In-class exercise: compare the result with what we'd get
# without error bars. Would the conclusions we draw from the
# data change? What about if we add in some less frequent
# outcomes, breeds, etc.?

# Statistical analysis
* Python is historically not widely used for traditional statistics
    * It's improving fast, but will probably never catch up to R
* But it has decent support for many common techniques

## Statistical tools in SciPy
* The SciPy package includes a well-developed [stats module](http://docs.scipy.org/doc/scipy/reference/stats.html)
* Caters more to computational scientists than end users
    * Interface is heavily object-oriented
    * You usually have to take the extra step yourself
    * Performance is very good, but interface is clunky

### Lots of descriptives and convenience functions

In [ ]:
# Descriptive statistics for age in months
print(stats.describe(data['age'].dropna()))

In [ ]:
# Plenty of little convenience functions
age = 60
percentile = stats.percentileofscore(data['age'], age)
print("\nA {}-month old animal is at the {:.0f}th percentile.".format(age, percentile)) 

### Distributions a-plenty
* Large number of statistical [distributions](http://docs.scipy.org/doc/scipy/reference/stats.html#continuous-distributions)
* All of the standard distribution functions (pdf, cdf, random sampling, etc.) 

In [ ]:
# Sample 100000 observations...
n = 100000

# ...from each of the normal, t(10), and beta(2, 5) distributions
normal = stats.norm.rvs(size=n)
t = stats.t.rvs(df=10, size=n)
beta = stats.beta.rvs(a=2, b=5, loc=-2, scale=6, size=n)

# Set up figure
ax = plt.figure(figsize=(6, 4)).gca()

# Plot a kde plot of each distribution
labels = ['t', 'normal', 'beta']
for i, dist in enumerate([t, normal, beta]):
    sns.kdeplot(dist, ax=ax, lw=2, label=labels[i])


In [ ]:
# Use CDF of normal to convert p-values to z-scores
print(stats.norm.ppf(0.001))

In [ ]:
# t statistic to p-value
print(stats.t.cdf(1.46, df=22))

In [ ]:
# Endpoints of interval containing 90% of distribution
print(stats.beta.interval(0.9, a=2, b=5))

### Basic tests
* You can do things like t-tests and ANOVAs with SciPy if you really want
* But they'd would be more pleasant in R—this isn't where Python shines

In [ ]:
# Independent samples t-test: do cats and dogs processed at the center differ in age?

# Extract 
dog_age = data.query('animal=="Dog"')['age'].dropna()
cat_age = data.query('animal=="Cat"')['age'].dropna()

# ttest_ind returns a tuple of t and p values
t, p = stats.ttest_ind(dog_age, cat_age)

# Report means as well
dm, cm = dog_age.mean(), cat_age.mean()

# Print something for the user
print("Dogs (mean = {:.2f}) vs. Cats (mean = {:.2f}), t = {:.2f}, p={:.2g}".format(dm, cm, t, p))

## [Statsmodels](http://statsmodels.sourceforge.net/devel/index.html#)
* Python's answer to base R
* It's not a great answer, but it's improving
* For common procedures, functionality is very similar to R

### What's in statsmodels
* Basic tests
* Linear models
* Diagnostics and specification tests
* Power calculations
* Multiple comparisons correction
* Time-series analysis
* Experimental support for many other things (e.g. mixed-effects models)

### Statsmodel examples
* Let's build a model to predict whether an animal is euthanized or not
* We'll use logistic regression using the formula interface
* We'll start by predicting euthanasia from just age, sex, and sterilization

In [ ]:
import statsmodels.formula.api as smf

# Drop missing values from analysis
tmp_data = data[['outcome', 'age', 'sex', 'sterilized']].dropna()
print("Kept {} observations.".format(len(tmp_data)))

# # The dependent variable: whether the animal was euthanized
tmp_data['y'] = (tmp_data['outcome'] == 'Euthanasia').astype(int)

# Specify the model with a formula
model = smf.logit('y ~ age + sex * sterilized', data=tmp_data)

# Fit model and save result
result = model.fit()

# Print a user-friendly summary
result.summary()

### Should you do classical statistics in Python?
* In the interest of providing a fair overview...
* If you plan to do a lot of classical inference (t-tests, ANOVAs, MLMs...), you will probably have an easier time in R
* If you want to do probabilistic programming, machine learning, or deep learning, Python shines
    * We'll cover some of this tomorrow